In [1]:
pwd

'/Users/sinayoks/dev/notebooks/little-book-of-semaphores'

In [2]:

#!mv /Users/sinayoks/Desktop/Screen\ Shot\ 2020-06-27\ at\ 14.49.39.png sync_app.png

# The Little Book of Semaphores

Working through my own implementations of concurrency and synchronization problems from the [Little Book of Semaphores](http://greenteapress.com/semaphores/LittleBookOfSemaphores.pdf) by A. B. Downey.

**Asyncio solutions in notebook**

We will implement examples using asyncio. 

**Reference solutions using Sync GUI app**

Solutions are also available from https://github.com/AllenDowney/LittleBookOfSemaphores/tree/master/code/sync_code and can be run using the Sync program provided in that repository:
```
git clone git@github.com:AllenDowney/LittleBookOfSemaphores.git
cd LittleBookOfSemaphores/code
python Sync.py sync_code/signal.py
```
![signalling problem using Downey's Sync app](sync_app.png)
```

```

In [3]:
import asyncio
from IPython.core.debugger import set_trace

In [4]:
async def run_test(test_coroutine, attempts):
    """Run a test multiple times to make sure we don't get lucky."""
    [await test_coroutine(attempt) for attempt in range(attempts)]

## Basic patterns

### Signaling
**Signaling problem**
2 threads/coroutines having to coordinate to do an action in a particular order.

In [23]:
async def test_signaling(attempt):
    """Check the signaling approach serialises the threads so that actionA 
    takes place before actionB. 
    
    Push a value from coroutines A then one from coroutine B into a shared queue, 
    and make sure they've been pushed in that order. 
    """
    # Not using a Lock here because we want to signal/release before waiting/acquiring 
    # so we need a semaphore. 
    first_action_done = asyncio.Semaphore(0)
    queue = asyncio.Queue()
    
    async def push(value):
        await queue.put(value)

    async def coroutineA():
        """Do first action then signal to B that we're done"""
        asyncio.Task.current_task().name = "coroutineA"
        await push('A')
        first_action_done.release()

    async def coroutineB():
        asyncio.Task.current_task().name = "coroutineB"
        async with first_action_done:
            res = await push('B')


    await asyncio.gather(coroutineA(), coroutineB())
    res = [await queue.get() for _ in range(queue.qsize())]
    assert res == ['A', 'B'], f'Test failed for attempt {attempt}: got {res}'

In [22]:
await run_test(test_signaling, 100)

### Rendez Vous
Two threads must await each other before doing some action. 

In [29]:
async def test_rendez_vous(attempt):
    """Make sure action doesn't happen before a rendez vous. 
    
    In this test, the action is reading the key value pairs from a shared dictionary. 
    Each of the two coroutines adds a key-value pair to the dictionary before the rendez vous. 
    So if both coroutines have waited for the other one successfully, they should both 
    return the two same (key,value) pairs. 
    """
    b_has_arrived = asyncio.Semaphore(0)
    a_has_arrived = asyncio.Semaphore(0)
    shared_dict = {}
    lock = asyncio.Lock()  # a lock to protect updating the dictionary
    
    async def read_items_from_dict():
        """Return tuple of (key, value) pairs giving the items in the dictionary."""
        async with lock:
            res = tuple(sorted(shared_dict.items()))
        return res 

    async def coroutineA(key, value):
        """Store key value pair in shared dictionary, wait for rendez vous with B, 
        then read all key value pairs from dictionary. 
        """
        a_has_arrived.release()
        # put in a value in the queue
        async with lock:
              shared_dict[key] = value
        await b_has_arrived.acquire()
        res = await read_items_from_dict()
        return res

    async def coroutineB(key, value):
        """Store key value pair in shared dictionary, wait for rendez vous with A, 
        then read all key value pairs from dictionary. 
        """
        b_has_arrived.release()
        async with lock:
              shared_dict[key] = value
        await a_has_arrived.acquire()
        res = await read_items_from_dict()
        return res

    itemsA, itemsB = await asyncio.gather(
        coroutineA('A', 1), 
        coroutineB('B', 2)
    )
    expected_items = ('A', 1), ('B', 2)
    results = {k:v for k,v in locals().items() if k in ['itemsA', 'itemsB', 'expected_items']}
    assert itemsA == itemsB == expected_items, results

In [30]:
await run_test(test_rendez_vous, 10)